In [ ]:
%load_ext autoreload
%autoreload 2
import sys
from ridge_utils.DataSequence import DataSequence
import pandas as pd
import os
import matplotlib.pyplot as plt
import cortex
import seaborn as sns
from os.path import join
from collections import defaultdict
import numpy as np
import sasc.viz
import joblib
import dvu
import sys
sys.path.append('../notebooks')
from tqdm import tqdm
from sasc.config import FMRI_DIR, STORIES_DIR, RESULTS_DIR
from neuro.config import repo_dir, PROCESSED_DIR
from neuro import analyze_helper, viz
from neuro.features.qa_questions import get_questions, get_merged_questions_v3_boostexamples
flatmaps_per_question = __import__('06_flatmaps_per_question')
from neurosynth import term_dict, term_dict_rev
import viz
from load_coef_flatmaps import _load_coefs_35questions, _load_coefs_individual, \
_load_coefs_individual_wordrate, _load_coefs_wordrate, _load_coefs_shapley

In [ ]:
results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/aug14_neurosynth_gemv'
rr_shapley, cols_varied, mets = analyze_helper.load_clean_results(results_dir)

In [ ]:
# results_dir = '/home/chansingh/mntv1/deep-fMRI/encoding/jun16_gpt4'
# rr, cols_varied, mets = analyze_helper.load_clean_results(results_dir)

In [ ]:
subject = 'S03'
df_w_shapley35 = _load_coefs_shapley(rr_shapley,
                                     subject=subject, qa_questions_version='v3_boostexamples_merged')

df_w_shapley_neurosynth = _load_coefs_shapley(rr_shapley,
                                              subject, qa_questions_version='v1neurosynth')

In [ ]:
# def _load_coefs_35_full(subject='S02'):
#     # # load questions model weights
#     # results_dir = analyze_helper.best_results_dir
#     # rr, cols_varied, mets = analyze_helper.load_clean_results(results_dir)
#     # metric_sort = 'corrs_tune_pc_weighted_mean'

#     # # pick model to interpret
#     # r = rr
#     # r = r[r.qa_questions_version == 'v3_boostexamples_merged']
#     # r = r[r.num_stories == -1]
#     # r = r[r.weight_enet_mask_num_nonzero == 35]
#     # r = r[r.feature_space == 'qa_embedder']
#     # cols_varied = [c for c in cols_varied if not c in ['num_stories',
#     #                                                    'feature_selection_alpha', 'feature_selection_stability_seeds']]
#     # args0 = r[r.subject == subject].iloc[0]
#     # weights, weights_pc = flatmaps_per_question.get_weights_top(args0)
#     # qs_selected = questions[args0['weight_enet_mask']]
#     # df_w_selected35 = pd.DataFrame({'question': qs_selected, 'weights': [
#     #                     w for w in weights]}).set_index('question')
#     # joblib.dump((args0, qs_selected, df_w_selected35),
#     # '../qa_results/processed/selected_weights.pkl')

#     args0, qs_selected, df_w_selected35 = joblib.load(
#         '../qa_results/processed/selected_weights.pkl')
#     return df_w_selected35

subject = 'S02'
r = rr_shapley
r = r[r.qa_questions_version == 'v3_boostexamples_merged']
r = r[r.weight_enet_mask_num_nonzero == 35]
r = r[r.feature_space == 'qa_embedder']
r = r[r.subject == subject]
r = r[r.use_random_subset_features == 0]

corrs_test = r['corrs_test']
joblib.dump(corrs_test, join(PROCESSED_DIR,
                             subject, 'corrs_test_35.pkl'))

In [ ]:
# subject = 'S02'
for subject in ['S01', 'S02', 'S03']:

    df_w_shapley_35 = _load_coefs_shapley(rr_shapley,
                                          subject, qa_questions_version='v3_boostexamples_merged')
    joblib.dump(df_w_shapley_35, join(
        PROCESSED_DIR, subject, 'shapley_35.pkl'))

    df_w_shapley_neurosynth = _load_coefs_shapley(rr_shapley,
                                                  subject, qa_questions_version='v1neurosynth')
    joblib.dump(df_w_shapley_neurosynth, join(PROCESSED_DIR,
                subject, 'shapley_neurosynth.pkl'))

    ########### use old models ###################
    # jointly fitted 35-question model
    # df_w_selected35 = _load_coefs_35questions(subject=subject)

    # individually fitted question models
    # df_w_individual = _load_coefs_individual(rr_shapley, subject=subject)
    # joblib.dump(df_w_individual, join(PROCESSED_DIR,
    # subject, 'individual.pkl'))

    # individually fitted question models *with wordrate
    # df_w_individual_wordrate = _load_coefs_individual_wordrate(
    # subject=subject)

    # wordrate
    # df_w_wordrate_alone = _load_coefs_wordrate(subject=subject)

    # # collate individual dfs #########################
    # # average weights for df_w_selected35 and df_w_individual
    # if subject == 'S02':
    #     df_avg = df_w_selected35.merge(df_w_individual, on='question')
    #     df_avg['weights'] = df_avg.apply(
    #         lambda x: np.mean([x['weights_x'], x['weights_y']], axis=0), axis=1)

    # df_avg_individual = df_w_individual.merge(
    #     df_w_individual_wordrate, on='question')
    # df_avg_individual['weights'] = df_avg_individual.apply(
    #     lambda x: np.mean([x['weights_x'], x['weights_y']], axis=0), axis=1)

    # df_qa_dict = {
    #     'selected35': df_w_selected35,
    #     'individual': df_w_individual,
    #     'individual_wordrate': df_w_individual_wordrate,
    #     'wordrate_alone': df_w_wordrate_alone,
    #     # 'avg': df_avg,
    #     'shapley_neurosynth': df_w_shapley_neurosynth,
    #     'shapley35': df_w_shapley35,
    #     'avg_individual': df_avg_individual
    # }
    # joblib.dump(df_qa_dict, f'df_qa_dict_{subject}.pkl')